## Project - my version

In [18]:
import numpy as np
import matplotlib.pyplot as plt

features = ['DER_mass_MMC','DER_mass_transverse_met_lep','DER_mass_vis','DER_pt_h','DER_deltaeta_jet_jet',
            'DER_mass_jet_jet','DER_prodeta_jet_jet','DER_deltar_tau_lep','DER_pt_tot','DER_sum_pt','DER_pt_ratio_lep_tau',
            'DER_met_phi_centrality','DER_lep_eta_centrality','PRI_tau_pt','PRI_tau_eta','PRI_tau_phi','PRI_lep_pt',
            'PRI_lep_eta','PRI_lep_phi','PRI_met','PRI_met_phi','PRI_met_sumet','PRI_jet_num','PRI_jet_leading_pt',
            'PRI_jet_leading_eta','PRI_jet_leading_phi','PRI_jet_subleading_pt','PRI_jet_subleading_eta',
            'PRI_jet_subleading_phi','PRI_jet_all_pt']

feats = list(zip(range(0,30,1),features))
print(feats)

[(0, 'DER_mass_MMC'), (1, 'DER_mass_transverse_met_lep'), (2, 'DER_mass_vis'), (3, 'DER_pt_h'), (4, 'DER_deltaeta_jet_jet'), (5, 'DER_mass_jet_jet'), (6, 'DER_prodeta_jet_jet'), (7, 'DER_deltar_tau_lep'), (8, 'DER_pt_tot'), (9, 'DER_sum_pt'), (10, 'DER_pt_ratio_lep_tau'), (11, 'DER_met_phi_centrality'), (12, 'DER_lep_eta_centrality'), (13, 'PRI_tau_pt'), (14, 'PRI_tau_eta'), (15, 'PRI_tau_phi'), (16, 'PRI_lep_pt'), (17, 'PRI_lep_eta'), (18, 'PRI_lep_phi'), (19, 'PRI_met'), (20, 'PRI_met_phi'), (21, 'PRI_met_sumet'), (22, 'PRI_jet_num'), (23, 'PRI_jet_leading_pt'), (24, 'PRI_jet_leading_eta'), (25, 'PRI_jet_leading_phi'), (26, 'PRI_jet_subleading_pt'), (27, 'PRI_jet_subleading_eta'), (28, 'PRI_jet_subleading_phi'), (29, 'PRI_jet_all_pt')]
